# Práctica 4: Procesamieto de audio

### **Participantes:**
- Gerardo León Quintana
- Susana Suárez Mendoza

## Ejercicio 1:

Construir un identificador de notas musicales. Es decir; en su versión más sencilla (y suficiente) la entrada es un sonido con una sola nota musical y debe identificar cuál es. Por simplicidad elija un único instrumento para la identificación.

## Ejercicio 2:

Construir una pequeña aplicación que permita operar con diferentes filtros (con un selector) y trabajar con varios umbrales. (uno para los filtros pasa-bajo y pasa-alto y dos para los filtros pasa-banda y rechaza-banda). Demuestre su funcionalidad con señales ruidosas. Muestre en cada filtrado la señal original y filtrada en el dominio temporal y en el dominio de la frecuencia.

# gerardo branch

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter, freqz
from scipy.io import wavfile
import tkinter as tk
from tkinter import filedialog
import vlc
from pydub import AudioSegment
import tempfile
import os
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg


In [3]:
def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandstop(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='bandstop')
    return b, a


In [4]:
# Función para aplicar un filtro
def apply_filter(data, b, a):
    y = lfilter(b, a, data)
    return y

# Función para plotear en el dominio temporal y de frecuencia
def plot_signal(time, signal, fs, title):
    plt.figure(figsize=(12, 6))
    
    # Dominio temporal
    plt.subplot(2, 1, 1)
    plt.plot(time, signal)
    plt.title(f'{title} - Time Domain')
    plt.xlabel('Time [s]')
    plt.ylabel('Amplitude')
    
    # Dominio de frecuencia
    plt.subplot(2, 1, 2)
    freqs = np.fft.fftfreq(len(signal), 1/fs)
    fft_spectrum = np.fft.fft(signal)
    plt.plot(freqs[:len(freqs)//2], np.abs(fft_spectrum)[:len(freqs)//2])
    plt.title(f'{title} - Frequency Domain')
    plt.xlabel('Frequency [Hz]')
    plt.ylabel('Amplitude')
    
    plt.tight_layout()
    plt.show()


In [ ]:

input_wav_file = 'crazy_frog_axel.wav'
fs, data = wavfile.read(input_wav_file)

# Normalizar si es necesario
if data.dtype == np.int16:
    data = data / 32768.0
elif data.dtype == np.int32:
    data = data / 2147483648.0

# Si el archivo de audio es estéreo, seleccionar un canal
if len(data.shape) == 2:
    data = data[:, 0]

# Crear vector de tiempo
t = np.arange(0, len(data) / fs, 1 / fs)

# Parámetros del filtro
order = 6

# Filtros y sus parámetros
filters = {
    'lowpass': {'func': butter_lowpass, 'params': {'cutoff': 1000}},
    'highpass': {'func': butter_highpass, 'params': {'cutoff': 1000}},
    'bandpass': {'func': butter_bandpass, 'params': {'lowcut': 500, 'highcut': 1500}},
    'bandstop': {'func': butter_bandstop, 'params': {'lowcut': 500, 'highcut': 1500}}
}

plot_signal(t, data, fs, 'Original Signal')

for filter in filters:
    b, a = filters[filter]['func'](fs=fs, order=order, **filters[filter]['params'])
    filtered_data = apply_filter(data, b, a)
    plot_signal(t, filtered_data, fs, f'Filtered Signal - {filter.capitalize()}')


In [9]:
# Clase para la interfaz gráfica
class AudioPlayer:
    def __init__(self, master):
        self.master = master
        self.master.title("Reproductor de Audio")
        self.master.geometry("300x150")

        # Inicializar el reproductor VLC
        self.player = None
        self.is_playing = False

        # Botón para elegir archivo
        self.select_button = tk.Button(self.master, text="Seleccionar Archivo", command=self.select_file)
        self.select_button.pack(pady=20)

        # Botón de reproducir/pausar
        self.play_button = tk.Button(self.master, text="Reproducir", command=self.toggle_play, state=tk.DISABLED)
        self.play_button.pack(pady=20)

    def select_file(self):
        # Abrir un cuadro de diálogo para seleccionar un archivo de audio
        file_path = filedialog.askopenfilename(title="Seleccionar Archivo de Audio", filetypes=[("Audio Files", "*.mp3;*.wav;*.ogg;*.flac")])
        if file_path:
            if self.player is not None and self.is_playing:
                self.player.stop()  # Detener el audio actual si está en reproducción

            self.player = vlc.MediaPlayer(file_path)
            self.play_button.config(state=tk.NORMAL)  # Habilitar el botón de reproducir
            self.is_playing = False  # Reiniciar estado de reproducción
            self.play_button.config(text="Reproducir")

    def toggle_play(self):
        if self.player is None:
            return  # No hay archivo seleccionado

        if self.is_playing:
            self.player.pause()
            self.play_button.config(text="Continuar")
        else:
            if self.player.is_playing():  # Si ya hay un audio en reproducción, detenerlo
                self.player.stop()  # Detener el audio actual
            self.player.play()  # Reproducir el nuevo audio
            self.play_button.config(text="Pausar")
        self.is_playing = not self.is_playing

# Crear la ventana principal
if __name__ == "__main__":
    root = tk.Tk()
    app = AudioPlayer(root)
    root.mainloop()


In [4]:
# Función para aplicar filtros
def apply_filter(data, filter_type, cutoff, fs, order=5, band=None):
    nyquist = 0.5 * fs
    if filter_type == 'low':
        normal_cutoff = cutoff / nyquist
        b, a = butter(order, normal_cutoff, btype='low', analog=False)
    elif filter_type == 'high':
        normal_cutoff = cutoff / nyquist
        b, a = butter(order, normal_cutoff, btype='high', analog=False)
    elif filter_type == 'band':
        low, high = band
        low /= nyquist
        high /= nyquist
        b, a = butter(order, [low, high], btype='band', analog=False)
    elif filter_type == 'notch':
        low, high = band
        low /= nyquist
        high /= nyquist
        b, a = butter(order, [low, high], btype='bandstop', analog=False)
    y = lfilter(b, a, data)
    return y

# Clase para la interfaz gráfica
class AudioPlayer:
    def __init__(self, master):
        self.master = master
        self.master.title("Reproductor de Audio")
        self.master.geometry("400x500")

        # Inicializar el reproductor VLC
        self.player = None
        self.is_playing = False

        # Botón para elegir archivo
        self.select_button = tk.Button(self.master, text="Seleccionar Archivo", command=self.select_file)
        self.select_button.pack(pady=10)

        # Opciones de filtros
        self.filter_type = tk.StringVar(value="none")
        self.cutoff = tk.DoubleVar(value=1000.0)
        self.low_cutoff = tk.DoubleVar(value=500.0)
        self.high_cutoff = tk.DoubleVar(value=2000.0)

        tk.Label(self.master, text="Seleccionar Filtro:").pack()
        filters = [("Ninguno", "none"), ("Pasa-Bajo", "low"), ("Pasa-Alto", "high"), ("Pasa-Banda", "band"), ("Rechaza-Banda", "notch")]
        for text, mode in filters:
            tk.Radiobutton(self.master, text=text, variable=self.filter_type, value=mode, command=self.apply_selected_filter).pack(anchor=tk.W)

        tk.Label(self.master, text="Frecuencia de Corte (Hz):").pack()
        self.cutoff_scale = tk.Scale(self.master, variable=self.cutoff, from_=20, to_=20000, orient=tk.HORIZONTAL, command=self.apply_selected_filter)
        self.cutoff_scale.pack()

        tk.Label(self.master, text="Frecuencia Baja (Hz):").pack()
        self.low_cutoff_scale = tk.Scale(self.master, variable=self.low_cutoff, from_=20, to_=20000, orient=tk.HORIZONTAL, command=self.apply_selected_filter)
        self.low_cutoff_scale.pack()

        tk.Label(self.master, text="Frecuencia Alta (Hz):").pack()
        self.high_cutoff_scale = tk.Scale(self.master, variable=self.high_cutoff, from_=20, to_=20000, orient=tk.HORIZONTAL, command=self.apply_selected_filter)
        self.high_cutoff_scale.pack()

        # Botón de reproducir/pausar
        self.play_button = tk.Button(self.master, text="Reproducir", command=self.toggle_play, state=tk.DISABLED)
        self.play_button.pack(pady=20)

        self.filtered_file = None  # Variable para almacenar el archivo filtrado temporal
        self.original_file = None  # Variable para almacenar el archivo original

    def select_file(self):
        # Abrir un cuadro de diálogo para seleccionar un archivo de audio
        file_path = filedialog.askopenfilename(title="Seleccionar Archivo de Audio", filetypes=[("Audio Files", "*.mp3;*.wav;*.ogg;*.flac")])
        if file_path:
            self.original_file = file_path
            self.apply_selected_filter()

    def apply_selected_filter(self, *args):
        if self.original_file:
            if self.player is not None and self.is_playing:
                self.player.pause()
                self.is_playing = False
                self.play_button.config(text="Reproducir")

            # Aplicar el filtro seleccionado al audio
            audio = AudioSegment.from_file(self.original_file)
            samples = np.array(audio.get_array_of_samples())
            fs = audio.frame_rate  # Tasa de muestreo

            filter_type = self.filter_type.get()
            cutoff = self.cutoff.get()
            low_cutoff = self.low_cutoff.get()
            high_cutoff = self.high_cutoff.get()
            if filter_type != "none":
                if filter_type in ["band", "notch"]:
                    filtered_samples = apply_filter(samples, filter_type, None, fs, band=(low_cutoff, high_cutoff))
                else:
                    filtered_samples = apply_filter(samples, filter_type, cutoff, fs)
                filtered_audio = audio._spawn(filtered_samples.astype(np.int16).tobytes())
            else:
                filtered_audio = audio

            # Guardar el audio filtrado en un archivo temporal
            with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as temp_file:
                filtered_audio.export(temp_file.name, format="wav")
                self.filtered_file = temp_file.name

            self.player = vlc.MediaPlayer(self.filtered_file)
            self.play_button.config(state=tk.NORMAL)  # Habilitar el botón de reproducir

    def toggle_play(self):
        if self.player is None:
            return  # No hay archivo seleccionado

        if self.is_playing:
            self.player.pause()
            self.play_button.config(text="Continuar")
        else:
            if self.player.is_playing():  # Si ya hay un audio en reproducción, detenerlo
                self.player.stop()  # Detener el audio actual
            self.player.play()  # Reproducir el nuevo audio
            self.play_button.config(text="Pausar")
        self.is_playing = not self.is_playing

    def __del__(self):
        if self.filtered_file:
            os.remove(self.filtered_file)  # Eliminar el archivo temporal al cerrar la aplicación

# Crear la ventana principal
if __name__ == "__main__":
    root = tk.Tk()
    app = AudioPlayer(root)
    root.mainloop()


In [3]:
# Función para aplicar filtros
def apply_filter(data, filter_type, cutoff, fs, order=5, band=None):
    nyquist = 0.5 * fs
    if filter_type == 'low':
        normal_cutoff = cutoff / nyquist
        b, a = butter(order, normal_cutoff, btype='low', analog=False)
    elif filter_type == 'high':
        normal_cutoff = cutoff / nyquist
        b, a = butter(order, normal_cutoff, btype='high', analog=False)
    elif filter_type == 'band':
        low, high = band
        low /= nyquist
        high /= nyquist
        b, a = butter(order, [low, high], btype='band', analog=False)
    elif filter_type == 'notch':
        low, high = band
        low /= nyquist
        high /= nyquist
        b, a = butter(order, [low, high], btype='bandstop', analog=False)
    y = lfilter(b, a, data)
    return y

# Clase para la interfaz gráfica
class AudioPlayer:
    def __init__(self, master):
        self.master = master
        self.master.title("Reproductor de Audio")
        self.master.geometry("800x600")

        # Inicializar el reproductor VLC
        self.player = None
        self.is_playing = False

        # Botón para elegir archivo
        self.select_button = tk.Button(self.master, text="Seleccionar Archivo", command=self.select_file)
        self.select_button.pack(pady=10)

        # Opciones de filtros
        self.filter_type = tk.StringVar(value="none")
        self.cutoff = tk.DoubleVar(value=1000.0)
        self.low_cutoff = tk.DoubleVar(value=500.0)
        self.high_cutoff = tk.DoubleVar(value=2000.0)

        tk.Label(self.master, text="Seleccionar Filtro:").pack()
        filters = [("Ninguno", "none"), ("Pasa-Bajo", "low"), ("Pasa-Alto", "high"), ("Pasa-Banda", "band"), ("Rechaza-Banda", "notch")]
        for text, mode in filters:
            tk.Radiobutton(self.master, text=text, variable=self.filter_type, value=mode, command=self.apply_selected_filter).pack(anchor=tk.W)

        tk.Label(self.master, text="Frecuencia de Corte (Hz):").pack()
        self.cutoff_scale = tk.Scale(self.master, variable=self.cutoff, from_=20, to_=20000, orient=tk.HORIZONTAL, command=self.apply_selected_filter)
        self.cutoff_scale.pack()

        tk.Label(self.master, text="Frecuencia Baja (Hz):").pack()
        self.low_cutoff_scale = tk.Scale(self.master, variable=self.low_cutoff, from_=20, to_=20000, orient=tk.HORIZONTAL, command=self.apply_selected_filter)
        self.low_cutoff_scale.pack()

        tk.Label(self.master, text="Frecuencia Alta (Hz):").pack()
        self.high_cutoff_scale = tk.Scale(self.master, variable=self.high_cutoff, from_=20, to_=20000, orient=tk.HORIZONTAL, command=self.apply_selected_filter)
        self.high_cutoff_scale.pack()

        # Botón de reproducir/pausar
        self.play_button = tk.Button(self.master, text="Reproducir", command=self.toggle_play, state=tk.DISABLED)
        self.play_button.pack(pady=20)

        # Botón para guardar archivo filtrado
        self.save_button = tk.Button(self.master, text="Guardar Archivo Filtrado", command=self.save_file, state=tk.DISABLED)
        self.save_button.pack(pady=10)

        self.filtered_file = None  # Variable para almacenar el archivo filtrado temporal
        self.original_file = None  # Variable para almacenar el archivo original

        # Control de volumen
        self.volume_var = tk.DoubleVar(value=100)
        self.volume_scale = tk.Scale(self.master, label="Volumen", variable=self.volume_var, from_=0, to_=100, orient=tk.HORIZONTAL, command=self.set_volume)
        self.volume_scale.pack(fill=tk.X, padx=10, pady=10)

        # Visualización de la señal de audio
        self.fig = Figure(figsize=(6, 2))
        self.ax = self.fig.add_subplot(111)
        self.canvas = FigureCanvasTkAgg(self.fig, master=self.master)
        self.canvas.get_tk_widget().pack(fill=tk.BOTH, expand=1)

    def select_file(self):
        # Abrir un cuadro de diálogo para seleccionar un archivo de audio
        file_path = filedialog.askopenfilename(title="Seleccionar Archivo de Audio", filetypes=[("Audio Files", "*.mp3;*.wav;*.ogg;*.flac")])
        if file_path:
            self.original_file = file_path
            self.apply_selected_filter()

    def apply_selected_filter(self, *args):
        if self.original_file:
            if self.player is not None and self.is_playing:
                self.player.pause()
                self.is_playing = False
                self.play_button.config(text="Reproducir")

            # Aplicar el filtro seleccionado al audio
            audio = AudioSegment.from_file(self.original_file)
            samples = np.array(audio.get_array_of_samples())
            fs = audio.frame_rate  # Tasa de muestreo

            filter_type = self.filter_type.get()
            cutoff = self.cutoff.get()
            low_cutoff = self.low_cutoff.get()
            high_cutoff = self.high_cutoff.get()
            if filter_type != "none":
                if filter_type in ["band", "notch"]:
                    filtered_samples = apply_filter(samples, filter_type, None, fs, band=(low_cutoff, high_cutoff))
                else:
                    filtered_samples = apply_filter(samples, filter_type, cutoff, fs)
                filtered_audio = audio._spawn(filtered_samples.astype(np.int16).tobytes())
            else:
                filtered_audio = audio

            # Guardar el audio filtrado en un archivo temporal
            with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as temp_file:
                filtered_audio.export(temp_file.name, format="wav")
                self.filtered_file = temp_file.name

            self.player = vlc.MediaPlayer(self.filtered_file)
            self.play_button.config(state=tk.NORMAL)  # Habilitar el botón de reproducir
            self.save_button.config(state=tk.NORMAL)  # Habilitar el botón de guardar

            # Actualizar la visualización de la señal de audio
            self.update_waveform(filtered_samples, fs)

    def toggle_play(self):
        if self.player is None:
            return  # No hay archivo seleccionado

        if self.is_playing:
            self.player.pause()
            self.play_button.config(text="Continuar")
        else:
            if self.player.is_playing():  # Si ya hay un audio en reproducción, detenerlo
                self.player.stop()  # Detener el audio actual
            self.player.play()  # Reproducir el nuevo audio
            self.play_button.config(text="Pausar")
        self.is_playing = not self.is_playing

    def set_volume(self, volume):
        if self.player:
            self.player.audio_set_volume(int(volume))

    def save_file(self):
        if self.filtered_file:
            save_path = filedialog.asksaveasfilename(title="filtered_audio", defaultextension=".wav",
                                                     filetypes=[("WAV Files", "*.wav")])
            if save_path:
                try:
                    with open(self.filtered_file, 'rb') as temp_file:
                        with open(save_path, 'wb') as out_file:
                            out_file.write(temp_file.read())
                    messagebox.showinfo("Guardar Archivo", "Archivo guardado exitosamente.")
                except Exception as e:
                    messagebox.showerror("Error", f"No se pudo guardar el archivo: {e}")

    def update_waveform(self, samples, fs):
        self.ax.clear()
        self.ax.plot(np.linspace(0, len(samples) / fs, num=len(samples)), samples)
        self.ax.set_xlabel("Tiempo [s]")
        self.ax.set_ylabel("Amplitud")
        self.canvas.draw()

    def __del__(self):
        if self.filtered_file:
            os.remove(self.filtered_file)  # Eliminar el archivo temporal al cerrar la aplicación

# Crear la ventana principal
if __name__ == "__main__":
    root = tk.Tk()
    app = AudioPlayer(root)
    root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\gerar\anaconda3\envs\aa1\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\gerar\AppData\Local\Temp\ipykernel_20852\869583925.py", line 88, in select_file
    self.apply_selected_filter()
  File "C:\Users\gerar\AppData\Local\Temp\ipykernel_20852\869583925.py", line 125, in apply_selected_filter
    self.update_waveform(filtered_samples, fs)
UnboundLocalError: local variable 'filtered_samples' referenced before assignment
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\gerar\anaconda3\envs\aa1\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\gerar\AppData\Local\Temp\ipykernel_20852\869583925.py", line 88, in select_file
    self.apply_selected_filter()
  File "C:\Users\gerar\AppData\Local\Temp\ipykernel_20852\869583925.py", line 125, in apply_selected_filter
    self.update_waveform(fi